<a href="https://colab.research.google.com/github/pneuly/whisper-asr-colab/blob/main/whisper_asr_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from logging import getLogger, basicConfig, StreamHandler, FileHandler, Formatter, INFO, WARNING, ERROR
import locale
from torch.cuda import empty_cache
from google.colab import files

locale.getpreferredencoding = lambda: "UTF-8"

# @title 自動文字起こし（Wisper）{ display-mode: "form" }
# @markdown 以下の設定項目を入力しセルを実行（Ctrl+Enter）<br>
# @markdown <font color="red">※文字の欠落が発生する場合はbatch_sizeを1にしてください。<br>
# @markdown 　設定項目の説明は下にあります。</font>
# @markdown #<b>設定</b>
audio = 'https://www.youtube.com/watch?v=xAmEQOqtMvA'  # @param {type:"string"}
model_size = "large-v3" # @param ["large-v3", "large-v2", "large", "medium", "small", "base", "tiny"] {allow-input: true}
diarization = True  # @param {type:"boolean"}
password = None  # @param {type:"string"}
start_time = None  # @param {type:"string"}
end_time = None  # @param {type:"string"}
timestamp_offset = None  # @param {type:"string"}
hotwords = "次に、これです。" # @param {type:"string"}
batch_size = 8 # @param {type:"number"}
realtime = False  # @param {type:"boolean"}
HUGGING_FACE_TOKEN = ""
initial_prompt = "定刻になりましたので、開始いたします。"
prefix = None
vad_filter = True,

# @markdown ###<br/><b>〔設定の説明〕</b>
# @markdown <b>audiopath:</b> 文字起こしする音声ファイルの場所<br/>
# @markdown 　　Youtubeの場合： https://www.youtube.com/......<br/>
# @markdown 　　手動で音声をアップロードした場合： 230401_1010.mp3 など<br/>
# @markdown 　　（アップロード完了まで待って実行してください）<br/>
# @markdown 　　<font color="red">空欄の場合はファイルアップロードボタンが表示されます</font>
# @markdown <br/><b>model_size:</b> 音声認識のモデルサイズ（mediumにすると少し精度が落ちるが早い）
# @markdown <br/><b>diarization:</b> 発言者別の文字起こしファイルを作成するか（Falseにすると早い）
# @markdown #### <br/><b><font color= "blue">以下は必要な場合のみ設定</font></b>
# @markdown <b>password:</b> パスワードを指定（Webexなど）</b>
# @markdown <br/><b>start_time:</b> 開始時間 hh:mm:ss</b>（指定しない場合は最初から）
# @markdown <br/><b>end_time:</b> 終了時間 hh:mm:ss（指定しない場合は最後まで）
# @markdown <br/><b>timestamp_offset:</b> タイムスタンプを指定の時間だけずらす hh:mm:ss（Noneの場合はstart_timeと連動）
# @markdown <br/><b>hotwords:</b> キーワード（次に、これです。は句読点を付けるために入れています。）
# @markdown <br/><b>batch_size:</b> 2以上にすると少し早くなります。欠落が発生する場合は1にしてください。
# @markdown <br/><b>reatime: </b><font color="red">ストリーミングをリアルタイムで文字起こしをする場合のみオンにしてください。</font>

# ----- main routine ------
basicConfig(
    stream=sys.stdout,
    level=WARNING,
    force=True)

if diarization and HUGGING_FACE_TOKEN == "":
    raise ValueError("HUGGING_FACE_TOKEN にトークンを入力してください")

if audio == "":
    audio = list(files.upload())[0]

!pip install git+https://github.com/pneuly/whisper-asr-colab.git@main -q
from whisper_asr_colab.worker import Worker  # noqa: E402

worker = Worker(
    audio=audio,
    model_size=model_size,
    language="ja",
    diarization=diarization,
    password=password,
    start_time=start_time,
    end_time=end_time,
    timestamp_offset=timestamp_offset,
    initial_prompt=initial_prompt,
    realtime=realtime,
    batch_size=batch_size,
    hugging_face_token=HUGGING_FACE_TOKEN,
    hotwords=hotwords,
    prefix=prefix,
    vad_filter=vad_filter
)

worker.run()

# gc GPU RAM
del worker
empty_cache()
